In [1]:
#本章要深入学习pandas的高级功能
#12.1分类数据
#背景和目的
#表中的一列通常回由重复的包含不同值的小集合的情况。我们已经学过了unique和value_counts,它们可以从数组提取出不同的值，并分别计算频率：

In [2]:
import numpy as np
import pandas as pd

In [3]:
values=pd.Series(['apple','orange','apple','apple']*2)

In [4]:
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [5]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [6]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

In [7]:
#许多数据系统（数据仓库、统计计算或其他应用）都发展出了特定的表征重复值方法，以进行高效的存储和计算。
#在数据仓库中，最好的方法是使用所谓饿包含不同值得维表，将主要得参数存储为引用维表整数键：
values=pd.Series([0,1,0,0]*2)

In [8]:
dim=pd.Series(['apple','orange'])

In [9]:
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [10]:
dim

0     apple
1    orange
dtype: object

In [11]:
#可以使用take方法存储原始得字符串Series
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

In [12]:
#这种用整数表示的方法称为分类或字典编码表示法。不同值得数组成为分裂、字典或数据级
#在本书中，我们使用分类得说法。表示分类的整数值称为分类编码或简单地称为编码
#分类表示可以在进行分析时大大提高性能。你也可以在保持编码不变的情况下，对分类进行转换。

In [13]:
#pandas的分类类型
#pandas有一个特殊的分类类型，用于保存使用整数分类表示法的数据。
fruits=['apple','orange','apple','apple']*2

In [14]:
N=len(fruits)

In [15]:
df=pd.DataFrame({'fruit':fruits,'basket_id':np.arange(N),'count':np.random.randint(3,15,size=N),'weight':np.random.uniform(0,4,size=N)},
                columns=['basket_id','fruit','count','weight'])

In [16]:
df

,basket_id,fruit,count,weight
0,0,apple,8,1.264338
1,1,orange,14,1.931997
2,2,apple,7,3.412945
3,3,apple,9,2.039982
4,4,apple,3,1.314709
5,5,orange,5,3.642719
6,6,apple,6,1.525828
7,7,apple,3,0.069876


In [17]:
fruit_cat=df['fruit'].astype('category')

In [18]:
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [19]:
c=fruit_cat.values

In [20]:
type(c)

pandas.core.arrays.categorical.Categorical

In [21]:
#分类对象有categories和caodes属性：
c.categories

Index(['apple', 'orange'], dtype='object')

In [22]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [23]:
df['fruit']=df['fruit'].astype('category')

In [24]:
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [25]:
#还可以从其他python序列直接创建pandas.Categorical
my_categories=pd.Categorical(['foo','bar','baz','foo','bar'])

In [26]:
my_categories

[foo, bar, baz, foo, bar]
Categories (3, object): [bar, baz, foo]

In [27]:
#如果你已经从其他源获得了分类编码，还可以使用from_codes构造器
categories=['foo','bar','baz']

In [28]:
codes=[0,1,2,0,0,1]

In [29]:
my_cats_2=pd.Categorical.from_codes(codes,categories)

In [30]:
my_cats_2

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

In [31]:
#与显示指定不同，分类变换不认定指定的分类顺序。因此取决于输入的数据的顺序，categories数组的顺序会不同。当使用from_codes或其他构造器时，可以指定分类一个有意义的顺序
ordered_cat=pd.Categorical.from_codes(codes,categories,ordered=True)

In [32]:
ordered_cat

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

In [33]:
my_cats_2.as_ordered()

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

In [34]:
#用分类进行计算
#某些pandas组件，比如groupby函数，更适合进行分类。还有一些函数可以使用有序标志位
np.random.seed(12345)

In [35]:
draws=np.random.randn(1000)

In [36]:
draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

In [37]:
#计算这个数据的分位面元，提取一些统计信息：
bins=pd.qcut(draws,4)
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

In [38]:
#虽然有用，确切的样本分位数与分位的名称相比，不利于生成汇总。我们可以使用labels参数qcut，实现目的：
bins=pd.qcut(draws,4,labels=['Q1','Q2','Q3','Q4'])
bins

[Q2, Q3, Q2, Q2, Q4, ..., Q3, Q2, Q1, Q3, Q4]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [39]:
bins.codes[:10]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

In [40]:
#加上标签的面元分类不包含数据面元边界的信息，因此可以使用groupby提取一些汇总信息：
bins=pd.Series(bins,name='quartile')
results=(pd.Series(draws).groupby(bins).agg(['count','min','max']).reset_index())

In [41]:
results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


In [42]:
#分位数列保存了原始的面元分类信息，包含排序：
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [43]:
#用分类提高性能
N=10000000

In [44]:
draws=pd.Series(np.random.randn(N))

In [45]:
labels=pd.Series(['foo','bar','baz','qux']*(N//4))

In [46]:
#将标签转换为分类
categories=labels.astype('category')

In [47]:
#这时可以看到标签使用的内存远比分类多：
labels.memory_usage()

80000080

In [48]:
categories.memory_usage()

10000272

In [49]:
#转换为分类不是没有代价的，但这是一次性的代价
%time _ =labels.astype('category')

Wall time: 2.1 s


In [50]:
#GroupBy使用分类操作明显更快，是因为底层的算法使用整数编码数组，而不是字符串数组
#分类方法
s=pd.Series(['a','b','c','d']*2)

In [51]:
cat_s=s.astype('category')

In [52]:
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

In [53]:
#特别的cat属性提供了分类方法的入口
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [54]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [55]:
#假设我们知道这个数据的实际分类集，超出了数据中的四个值。我们可以使用set_categories改变他们

In [56]:
actural_categories=['a','b','c','d','e']

In [59]:
cat_s2=cat_s.cat.set_categories(actural_categories)
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]

In [60]:
cat_s.value_counts()

d    2
c    2
b    2
a    2
dtype: int64

In [61]:
cat_s2.value_counts()

d    2
c    2
b    2
a    2
e    0
dtype: int64

In [62]:
#在大数据集中，分类经常作为节省内存和高性能的便捷工具。
#过滤完大DataFrame或Series之后，许多分类可能不会出现在数据中。
#我们可以使用remove_unused_categories方法删除没看到的分类
cat_s3=cat_s[cat_s.isin(['a','b'])]

In [63]:
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): [a, b, c, d]

In [64]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): [a, b]

In [65]:
#为建模创建虚拟变量
cat_s=pd.Series(['a','b','c','d']*2,dtype='category')

In [66]:
#pandas.get_dummies函数可以转换这个分类数据为包含虚拟变量的DataFrame
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


In [69]:
#分组转换和‘解封’GroupBy
df=pd.DataFrame({'key':['a','b','c']*4,'value':np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [70]:
g=df.groupby('key').value

In [71]:
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [72]:
#假设产生一个和df['value']形状相同的Series，但值替换为按键分组的平均值
#我们可以传递lambda x：x.mean()进行转换
g.transform(lambda x: x.mean())

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [73]:
#对于内置的聚合函数，我们可以传递一个字符串假名作为GroupBy的agg方法
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [74]:
#与apply类似，transfrom的函数会返回Series，但是结果必须与输入大小相同。
g.transform(lambda x: x*2)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [75]:
g.transform(lambda x:x.rank(ascending=False))

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [76]:
#看一个简单聚合构造的分组转换函数：
def normalize(x):
    return (x-x.mean())/x.std()

In [77]:
#我们用transfrom或apply可以获得等价的结果
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [78]:
g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [80]:
#内置的聚合函数，比如mean或sum，通常比apply函数快，也比transform快，
#这允许我们进行一个所谓的解封分组操作。
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [81]:
normalized=(df['value']-g.transform('mean'))/g.transform('std')

In [82]:
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [83]:
#分组的时间重采样
#对于时间序列数据，resample方法从语义上是一个基于内在时间的分组操作
N=15
times=pd.date_range('2017-05-20 00:00',freq='1min',periods=N)

In [85]:
df=pd.DataFrame({'time':times,'value':np.arange(N)})

In [86]:
df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


In [87]:
#对time作为索引，然后重采样
df.set_index('time').resample('5min').count()

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


In [88]:
#假设DataFrame包含多个时间序列，用一个额外的分组键的列进行标记
df2=pd.DataFrame({'time':times.repeat(3),'key':np.tile(['a','b','c'],N),'value':np.arange(N*3.)})

In [89]:
df2[:7]

,time,key,value
0,2017-05-20 00:00:00,a,0.0
1,2017-05-20 00:00:00,b,1.0
2,2017-05-20 00:00:00,c,2.0
3,2017-05-20 00:01:00,a,3.0
4,2017-05-20 00:01:00,b,4.0
5,2017-05-20 00:01:00,c,5.0
6,2017-05-20 00:02:00,a,6.0


In [90]:
#要对每个key进行相同的重采样，我们引入pandas.TimeGrouper对象：
time_key=pd.TimeGrouper('5min')

D:\Anacaonda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  


In [91]:
resampled=(df2.set_index('time').groupby(['key',time_key]).sum())

In [92]:
resampled

value
key time                      
a   2017-05-20 00:00:00   30.0
    2017-05-20 00:05:00  105.0
    2017-05-20 00:10:00  180.0
b   2017-05-20 00:00:00   35.0
    2017-05-20 00:05:00  110.0
    2017-05-20 00:10:00  185.0
c   2017-05-20 00:00:00   40.0
    2017-05-20 00:05:00  115.0
    2017-05-20 00:10:00  190.0

In [93]:
resampled.reset_index()

,key,time,value
0,a,2017-05-20 00:00:00,30.0
1,a,2017-05-20 00:05:00,105.0
2,a,2017-05-20 00:10:00,180.0
3,b,2017-05-20 00:00:00,35.0
4,b,2017-05-20 00:05:00,110.0
5,b,2017-05-20 00:10:00,185.0
6,c,2017-05-20 00:00:00,40.0
7,c,2017-05-20 00:05:00,115.0
8,c,2017-05-20 00:10:00,190.0


In [97]:
#12.3链式编程技术
#管道方法

In [96]:
#12.4总结

KeyError: 'key1'